In [7]:
# Binomial Options Pricing Model

import numpy as np

def binomial_option_pricing(S, K, r, v, T, N, option_type='call'):
    """
    Calculate the option price using the binomial options pricing model.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    r (float): Risk-free interest rate
    v (float): Volatility of the stock
    T (float): Time to expiration (in years)
    N (int): Number of steps in the binomial tree
    option_type (str): Type of the option ('call' or 'put')

    Returns:
    float: Estimated option price
    """
    # Time step
    dt = T / N
    # Up and down factors
    u = np.exp(v * np.sqrt(dt))
    d = 1 / u
    # Risk-neutral probability
    q = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(N + 1)
    ST[0] = S * d**N
    for i in range(1, N + 1):
        ST[i] = ST[i - 1] * u / d

    # Initialize option values at maturity
    if option_type == 'call':
        option_values = np.maximum(ST - K, 0)
    else:
        option_values = np.maximum(K - ST, 0)

    # Step back through the tree
    for i in range(N - 1, -1, -1):
        option_values = (q * option_values[1:] + (1 - q) * option_values[:-1]) * np.exp(-r * dt)

    return option_values[0]

# Example usage
current_stock_price = 100
strike_price = 100
risk_free_rate = 0.05
volatility = 0.2
time_to_expiration = 1  # 1 year
steps = 100

option_price = binomial_option_pricing(current_stock_price, strike_price, risk_free_rate, volatility, time_to_expiration, steps, 'call')
print(f"Call Option Price: {option_price:.2f}")

Call Option Price: 10.43


In [9]:
# Black-Scholes Options Pricing Model

import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calculate the European option price using the Black-Scholes formula.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    r (float): Risk-free interest rate
    sigma (float): Volatility of the stock
    T (float): Time to expiration (in years)
    option_type (str): Type of the option ('call' or 'put')

    Returns:
    float: Estimated option price
    """
    # Black-Scholes d1 and d2 calculations
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    return option_price

# Example usage
current_stock_price = 100
strike_price = 100
risk_free_rate = 0.05
volatility = 0.2
time_to_expiration = 1  # 1 year

call_option_price = black_scholes(current_stock_price, strike_price, time_to_expiration, risk_free_rate, volatility, "call")
put_option_price = black_scholes(current_stock_price, strike_price, time_to_expiration, risk_free_rate, volatility, "put")

print(f"Call Option Price: {call_option_price:.2f}")
print(f"Put Option Price: {put_option_price:.2f}")


Call Option Price: 10.45
Put Option Price: 5.57


In [10]:
# Monte-Carlo Simulation

import numpy as np

def monte_carlo_option_pricing(S, K, T, r, sigma, N, option_type="call"):
    """
    Calculate the European option price using a Monte Carlo simulation.

    Parameters:
    S (float): Current stock price
    K (float): Strike price
    r (float): Risk-free interest rate
    sigma (float): Volatility of the stock
    T (float): Time to expiration (in years)
    N (int): Number of simulations
    option_type (str): Type of the option ('call' or 'put')

    Returns:
    float: Estimated option price
    """
    payoffs = np.zeros(N)
    np.random.seed(0)  # For reproducible results

    for i in range(N):
        # Simulate a path ending price
        Z = np.random.normal(0, 1)
        ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

        # Calculate the payoff for call/put
        if option_type == "call":
            payoffs[i] = max(ST - K, 0)
        elif option_type == "put":
            payoffs[i] = max(K - ST, 0)
        else:
            raise ValueError("Invalid option type. Use 'call' or 'put'.")

    # Discount payoffs back to present value and average
    option_price = np.exp(-r * T) * np.mean(payoffs)

    return option_price

# Example usage
current_stock_price = 100
strike_price = 100
risk_free_rate = 0.05
volatility = 0.2
time_to_expiration = 1  # 1 year
num_simulations = 10000

call_option_price = monte_carlo_option_pricing(current_stock_price, strike_price, time_to_expiration, risk_free_rate, volatility, num_simulations, "call")
put_option_price = monte_carlo_option_pricing(current_stock_price, strike_price, time_to_expiration, risk_free_rate, volatility, num_simulations, "put")

print(f"Call Option Price: {call_option_price:.2f}")
print(f"Put Option Price: {put_option_price:.2f}")


Call Option Price: 10.08
Put Option Price: 5.62


In [11]:
import tkinter as tk
from tkinter import ttk


# GUI application

class OptionPricingApp:
    def __init__(self, root):
        self.root = root
        root.title("Options Pricing Models")

        # Create and place labels, entries, and buttons
        self.create_widgets()

    def create_widgets(self):
        # Labels and entries for inputs
        inputs = ['Current Stock Price (S)', 'Strike Price (K)', 'Risk-free Rate (r)', 'Volatility (σ)', 'Time to Expiration (T)', 'Steps/Simulations (N)']
        self.entries = {}
        for i, label in enumerate(inputs):
            ttk.Label(self.root, text=label).grid(row=i, column=0)
            entry = ttk.Entry(self.root)
            entry.grid(row=i, column=1)
            self.entries[label] = entry

        # Option type selection
        self.option_type = tk.StringVar()
        ttk.Radiobutton(self.root, text='Call', variable=self.option_type, value='call').grid(row=7, column=0)
        ttk.Radiobutton(self.root, text='Put', variable=self.option_type, value='put').grid(row=7, column=1)
        self.option_type.set('call')  # set default value

        # Buttons for each model
        ttk.Button(self.root, text="Binomial Model", command=self.calculate_binomial).grid(row=8, column=0)
        ttk.Button(self.root, text="Black-Scholes Model", command=self.calculate_black_scholes).grid(row=8, column=1)
        ttk.Button(self.root, text="Monte Carlo Model", command=self.calculate_monte_carlo).grid(row=8, column=2)

        # Output display
        self.result_var = tk.StringVar()
        ttk.Label(self.root, textvariable=self.result_var).grid(row=9, columnspan=3)

    def calculate_binomial(self):
        # Extract values and call the binomial pricing function
        values = {k: float(v.get()) for k, v in self.entries.items()}
        option_price = binomial_option_pricing(values['Current Stock Price (S)'], values['Strike Price (K)'],
                                               values['Risk-free Rate (r)'], values['Volatility (σ)'],
                                               values['Time to Expiration (T)'], int(values['Steps/Simulations (N)']),
                                               self.option_type.get())
        self.result_var.set(f"Binomial Model Price: {option_price:.2f}")

    def calculate_black_scholes(self):
        # Extract values and call the Black-Scholes pricing function
        values = {k: float(v.get()) for k, v in self.entries.items()}
        option_price = black_scholes(values['Current Stock Price (S)'], values['Strike Price (K)'],
                                     values['Time to Expiration (T)'], values['Risk-free Rate (r)'],
                                     values['Volatility (σ)'], self.option_type.get())
        self.result_var.set(f"Black-Scholes Model Price: {option_price:.2f}")

    def calculate_monte_carlo(self):
        # Extract values and call the Monte Carlo pricing function
        values = {k: float(v.get()) for k, v in self.entries.items()}
        option_price = monte_carlo_option_pricing(values['Current Stock Price (S)'], values['Strike Price (K)'],
                                                  values['Time to Expiration (T)'], values['Risk-free Rate (r)'],
                                                  values['Volatility (σ)'], int(values['Steps/Simulations (N)']),
                                                  self.option_type.get())
        self.result_var.set(f"Monte Carlo Model Price: {option_price:.2f}")

# Main function to run the application
def main():
    root = tk.Tk()
    app = OptionPricingApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()